In [1]:
import pandas as pd   #to create dataframe
import requests       #to send the request to the URL
from bs4 import BeautifulSoup #to get the content in the form of HTML
import numpy as np
import imdb 
import nltk
import seaborn

In [2]:
film_name=input('het film')
ia = imdb.IMDb()

search = ia.search_movie(film_name)
id = search[0].movieID


het filmThe Godfather


In [3]:
print(id) 

0068646


In [4]:
comments = []
rating_per_comment = []
for i in range(10):
    #assigning the URL with variable name url
    url = 'https://www.imdb.com/title/tt'+id+'/reviews?sort=curated&dir=desc&ratingFilter={}'.format(i)
    #request allow you to send HTTP request
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    #movie_data : comments
    movie_data = soup.findAll('a', attrs= {'class': 'title'})
    #rating : rating of the comments
    rating = soup.findAll('span' , attrs= {'class' : 'rating-other-user-rating'})
    for ratings in rating:
        rating_per_comments = ratings.span.text
        rating_per_comment.append(rating_per_comments)
    for comment in movie_data:   
        comments.append(comment.text.replace('\n',''))

In [5]:
review = pd.DataFrame({'comment' : comments})

In [6]:
review.head()

,comment
0,The Pinnacle Of Flawless Films!
1,"An offer so good, I couldn't refuse"
2,"For Me, This Is The Definitive Film"
3,An Iconic Film
4,The greatest movie of all time!


In [7]:
review.reset_index().rename(columns={'index':'id'})

,id,comment
0,0,The Pinnacle Of Flawless Films!
1,1,"An offer so good, I couldn't refuse"
2,2,"For Me, This Is The Definitive Film"
3,3,An Iconic Film
4,4,The greatest movie of all time!
...,...,...
244,244,This movie is amazing!!!!!
245,245,In the dictionary under masterpiece
246,246,not the godfather of all movies
247,247,IT'S JUST TOO LONG


In [8]:
review.shape

(249, 1)

In [9]:
exemple=review['comment'][0]
tokens=nltk.word_tokenize(exemple)

In [10]:
nltk.pos_tag(tokens)

[('The', 'DT'),
 ('Pinnacle', 'NNP'),
 ('Of', 'IN'),
 ('Flawless', 'NNP'),
 ('Films', 'NNP'),
 ('!', '.')]

In [11]:
from nltk.sentiment import SentimentIntensityAnalyzer 
from tqdm.notebook import tqdm
sia=SentimentIntensityAnalyzer()

In [12]:
sia.polarity_scores(exemple)

{'neg': 0.0, 'neu': 0.527, 'pos': 0.473, 'compound': 0.5562}

In [13]:
res={}
for i,row in tqdm(review.iterrows(),total=len(review)):
    text=row['comment']
    res[i]=sia.polarity_scores(text)
    

  0%|          | 0/249 [00:00<?, ?it/s]

In [14]:
res

{0: {'neg': 0.0, 'neu': 0.527, 'pos': 0.473, 'compound': 0.5562},
 1: {'neg': 0.0, 'neu': 0.404, 'pos': 0.596, 'compound': 0.7089},
 2: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 3: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 4: {'neg': 0.0, 'neu': 0.527, 'pos': 0.473, 'compound': 0.6696},
 5: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 6: {'neg': 0.0, 'neu': 0.878, 'pos': 0.122, 'compound': 0.2714},
 7: {'neg': 0.099, 'neu': 0.634, 'pos': 0.268, 'compound': 0.5267},
 8: {'neg': 0.0, 'neu': 0.588, 'pos': 0.412, 'compound': 0.6369},
 9: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 10: {'neg': 0.0, 'neu': 0.507, 'pos': 0.493, 'compound': 0.8934},
 11: {'neg': 0.305, 'neu': 0.695, 'pos': 0.0, 'compound': -0.3875},
 12: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 13: {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.5719},
 14: {'neg': 0.337, 'neu': 0.288, 'pos': 0.375, 'compound': 0.1027},
 15: {'neg': 0.0, 'neu': 1.0, 'pos': 0.

In [15]:
pd.DataFrame(res).T
vaders=pd.DataFrame(res).T

In [16]:
vader=vaders.reset_index().rename(columns={'index':'id'})

In [17]:
vader

,id,neg,neu,pos,compound
0,0,0.0,0.527,0.473,0.5562
1,1,0.0,0.404,0.596,0.7089
2,2,0.0,1.000,0.000,0.0000
3,3,0.0,1.000,0.000,0.0000
4,4,0.0,0.527,0.473,0.6696
...,...,...,...,...,...
244,244,0.0,1.000,0.000,0.0000
245,245,0.0,0.494,0.506,0.6249
246,246,0.0,1.000,0.000,0.0000
247,247,0.0,1.000,0.000,0.0000


In [18]:
review

,comment
0,The Pinnacle Of Flawless Films!
1,"An offer so good, I couldn't refuse"
2,"For Me, This Is The Definitive Film"
3,An Iconic Film
4,The greatest movie of all time!
...,...
244,This movie is amazing!!!!!
245,In the dictionary under masterpiece
246,not the godfather of all movies
247,IT'S JUST TOO LONG


In [20]:
res=review.join(vader['compound'])

In [21]:
res

,comment,compound
0,The Pinnacle Of Flawless Films!,0.5562
1,"An offer so good, I couldn't refuse",0.7089
2,"For Me, This Is The Definitive Film",0.0000
3,An Iconic Film,0.0000
4,The greatest movie of all time!,0.6696
...,...,...
244,This movie is amazing!!!!!,0.0000
245,In the dictionary under masterpiece,0.6249
246,not the godfather of all movies,0.0000
247,IT'S JUST TOO LONG,0.0000


In [23]:
res.to_csv("resultat.csv")